# Callflow-Tracer Jupyter Notebook Examples

This notebook demonstrates how to use callflow-tracer in Jupyter notebooks for interactive call graph visualization and profiling.

## Setup

First, let's import the necessary modules and initialize the Jupyter integration.

In [1]:
import sys
import os
import time
import numpy as np

# Add the package to path if running from examples directory
sys.path.insert(0, os.path.abspath('..'))

from callflow_tracer import trace, trace_scope, profile_function, profile_section
from callflow_tracer.jupyter import display_callgraph, init_jupyter
from callflow_tracer.exporter import export_html

# Initialize Jupyter integration
init_jupyter()

print("✓ Callflow-tracer loaded successfully!")

Callflow Tracer Jupyter integration loaded. Use %callflow_trace or %%callflow_cell_trace
✓ Callflow-tracer loaded successfully!


## Example 1: Basic Function Tracing

Let's create some simple functions and trace their execution.

In [2]:
def calculate_sum(numbers):
    """Calculate sum of numbers."""
    return sum(numbers)

def calculate_average(numbers):
    """Calculate average of numbers."""
    total = calculate_sum(numbers)
    return total / len(numbers)

def process_data(data):
    """Process data and return statistics."""
    avg = calculate_average(data)
    total = calculate_sum(data)
    return {'average': avg, 'total': total}

# Trace the execution
with trace_scope() as graph:
    result = process_data([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
    print(f"Result: {result}")

# Display the call graph
display_callgraph(graph.to_dict(), height="400px", layout="hierarchical")

print(f"\nGraph captured {len(graph.nodes)} nodes and {len(graph.edges)} edges")

Result: {'average': 5.5, 'total': 55}


NameError: name 'nodes' is not defined

## Example 2: Recursive Function Tracing

Trace recursive functions to see the call hierarchy.

In [3]:
def fibonacci(n):
    """Calculate fibonacci number recursively."""
    if n <= 1:
        return n
    return fibonacci(n - 1) + fibonacci(n - 2)

def factorial(n):
    """Calculate factorial recursively."""
    if n <= 1:
        return 1
    return n * factorial(n - 1)

# Trace recursive calls
with trace_scope() as graph:
    fib_result = fibonacci(8)
    fact_result = factorial(5)
    print(f"Fibonacci(8) = {fib_result}")
    print(f"Factorial(5) = {fact_result}")

# Display with force-directed layout
display_callgraph(graph.to_dict(), height="500px", layout="force")

print(f"\nCaptured {len(graph.nodes)} function calls")

Fibonacci(8) = 21
Factorial(5) = 120


NameError: name 'nodes' is not defined

## Example 3: Performance Profiling

Use profiling to measure execution time, memory usage, and CPU performance.

In [ ]:
@profile_function
def matrix_multiplication(size):
    """Perform matrix multiplication."""
    a = np.random.rand(size, size)
    b = np.random.rand(size, size)
    return np.dot(a, b)

@profile_function
def data_processing(n):
    """Process large dataset."""
    data = [i ** 2 for i in range(n)]
    result = sum(data) / len(data)
    return result

# Run with profiling
print("Running matrix multiplication...")
result1 = matrix_multiplication(100)
print(f"Matrix shape: {result1.shape}")

print("\nRunning data processing...")
result2 = data_processing(10000)
print(f"Average: {result2:.2f}")

# Display profiling stats
print("\n=== Matrix Multiplication Stats ===")
stats1 = matrix_multiplication.performance_stats
print(f"I/O wait time: {stats1.io_wait_time:.4f}s")
mem_stats1 = stats1._get_memory_stats()
if mem_stats1:
    print(f"Memory: {mem_stats1['current_mb']:.2f}MB (peak: {mem_stats1['peak_mb']:.2f}MB)")

print("\n=== Data Processing Stats ===")
stats2 = data_processing.performance_stats
print(f"I/O wait time: {stats2.io_wait_time:.4f}s")
mem_stats2 = stats2._get_memory_stats()
if mem_stats2:
    print(f"Memory: {mem_stats2['current_mb']:.2f}MB (peak: {mem_stats2['peak_mb']:.2f}MB)")

## Example 4: Combined Tracing and Profiling

Combine call graph tracing with performance profiling for comprehensive analysis.

In [ ]:
def load_data(size):
    """Simulate data loading."""
    time.sleep(0.05)  # Simulate I/O
    return np.random.rand(size)

def transform_data(data):
    """Transform data."""
    return data * 2 + 1

def analyze_data(data):
    """Analyze transformed data."""
    return {
        'mean': np.mean(data),
        'std': np.std(data),
        'min': np.min(data),
        'max': np.max(data)
    }

def pipeline(size):
    """Complete data pipeline."""
    data = load_data(size)
    transformed = transform_data(data)
    results = analyze_data(transformed)
    return results

# Run with both tracing and profiling
with profile_section("Data Pipeline") as perf_stats:
    with trace_scope() as graph:
        results = pipeline(1000)
        print(f"Analysis Results: {results}")

# Display call graph
display_callgraph(graph.to_dict(), height="400px", layout="hierarchical")

# Display profiling stats
print("\n=== Performance Stats ===")
stats_dict = perf_stats.to_dict()
print(f"I/O wait time: {stats_dict['io_wait']:.4f}s")
if stats_dict['memory']:
    print(f"Memory: {stats_dict['memory']['current_mb']:.2f}MB")
if stats_dict['cpu']:
    print("\nCPU Profile (first 300 chars):")
    print(stats_dict['cpu']['profile_data'][:300])

## Example 5: Export to HTML

Export the call graph with profiling data to an interactive HTML file.

In [ ]:
# Create a complex workflow
def step1():
    time.sleep(0.01)
    return "Step 1 complete"

def step2():
    time.sleep(0.02)
    return "Step 2 complete"

def step3():
    result1 = step1()
    result2 = step2()
    return f"{result1}, {result2}"

def main_workflow():
    return step3()

# Trace and profile
with profile_section("Complete Workflow") as perf_stats:
    with trace_scope() as graph:
        result = main_workflow()
        print(result)

# Export to HTML
output_file = "jupyter_callgraph_export.html"
export_html(
    graph, 
    output_file,
    title="Jupyter Notebook Call Graph",
    profiling_stats=perf_stats.to_dict()
)

print(f"\n✓ Exported to {output_file}")
print(f"  - {len(graph.nodes)} nodes")
print(f"  - {len(graph.edges)} edges")
print(f"  - Profiling data included")

## Example 6: Using Magic Commands

Use Jupyter magic commands for quick tracing.

In [ ]:
%%callflow_cell_trace

def quick_test(n):
    """Quick test function."""
    return sum(range(n))

def another_function(n):
    """Another function."""
    result = quick_test(n)
    return result * 2

# This entire cell will be traced
result = another_function(100)
print(f"Result: {result}")

## Example 7: Real-World Scenario - Data Science Pipeline

A realistic data science workflow with multiple stages.

In [ ]:
def fetch_dataset(n_samples):
    """Simulate fetching data from a source."""
    time.sleep(0.02)
    return np.random.rand(n_samples, 5)

def preprocess(data):
    """Preprocess the data."""
    # Normalize
    normalized = (data - data.mean(axis=0)) / data.std(axis=0)
    return normalized

def feature_engineering(data):
    """Create new features."""
    # Add polynomial features
    squared = data ** 2
    return np.hstack([data, squared])

def train_model(features):
    """Simulate model training."""
    time.sleep(0.05)
    # Simple computation to simulate training
    weights = np.mean(features, axis=0)
    return weights

def evaluate_model(weights, features):
    """Evaluate model performance."""
    predictions = np.dot(features, weights)
    score = np.mean(predictions)
    return score

def ml_pipeline(n_samples):
    """Complete ML pipeline."""
    # Fetch data
    data = fetch_dataset(n_samples)
    
    # Preprocess
    processed = preprocess(data)
    
    # Feature engineering
    features = feature_engineering(processed)
    
    # Train
    model = train_model(features)
    
    # Evaluate
    score = evaluate_model(model, features)
    
    return score

# Run the pipeline with tracing and profiling
print("Running ML Pipeline...\n")

with profile_section("ML Pipeline") as perf_stats:
    with trace_scope() as graph:
        final_score = ml_pipeline(500)
        print(f"Model Score: {final_score:.4f}")

# Visualize the pipeline
display_callgraph(graph.to_dict(), height="600px", layout="hierarchical")

# Show performance metrics
print("\n=== Pipeline Performance ===")
stats = perf_stats.to_dict()
print(f"Total I/O wait: {stats['io_wait']:.4f}s")
if stats['memory']:
    print(f"Peak memory: {stats['memory']['peak_mb']:.2f}MB")
print(f"Functions called: {len(graph.nodes)}")
print(f"Call relationships: {len(graph.edges)}")

## Summary

This notebook demonstrated:

1. **Basic tracing** - Capture function call relationships
2. **Recursive functions** - Visualize complex call hierarchies
3. **Performance profiling** - Measure execution time and memory
4. **Combined analysis** - Trace + profile together
5. **HTML export** - Save interactive visualizations
6. **Magic commands** - Quick tracing with IPython magics
7. **Real-world example** - Complete ML pipeline analysis

### Key Features:

- 📊 Interactive call graph visualization
- ⚡ Performance profiling (CPU, memory, I/O)
- 🔄 Multiple layout options (hierarchical, force-directed, circular, timeline)
- 💾 Export to standalone HTML files
- 🪄 Jupyter magic commands for convenience
- 🎯 Detailed function call statistics